In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate, accuracy
import re

bs = 64

# fixes fastai + pandas error
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [3]:
path = "data/"
df = pd.read_csv(f"{path}train.csv", low_memory=False)
df.head()

,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3


In [4]:
# Add value for sightings to ease creation of validation set
sighting_count = df[df['Id'] != 'new_whale'].Id.value_counts()
sighting_count.name = 'sightings'
df = df.join(sighting_count, on='Id')
df.head()

,Image,Id,sightings
0,00022e1a.jpg,w_e15442c,1.0
1,000466c4.jpg,w_1287fbc,34.0
2,00087b01.jpg,w_da2efe0,11.0
3,001296d5.jpg,w_19e5482,1.0
4,0014cfdf.jpg,w_f22f3e3,2.0


In [5]:
val_func = set(
    df.sample(frac=1)[(df['Id'] != 'new_whale') & (df['sightings'] > 1)].groupby('Id').first().Image
)

# add 700 new whales to the validation set
val_func = val_func.union(
    set(df[df['Id'] == 'new_whale'].sample(frac=1).Image.values[:700])
)

C:\Users\matt\Miniconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [6]:
func_to_label = { row[1].Image: row[1].Id for row in df.iterrows() }

path_to_func = lambda path: re.search('\w*\.jpg$', path).group(0)

In [9]:
df_val = df[df.Image.isin(val_func)]
df_train = df[~df.Image.isin(val_func)]
df_train_val = df

In [12]:
result = None
sample_size = 15

for group in df_train_val.groupby('Id'):
    n = group[1].shape[0]
    new_rows = group[1].sample(0 if sample_size < n else sample_size - n, replace=True)
    rows = pd.concat((group[1], new_rows))
    
    if result is None:
        result = rows
    else:
        result = pd.concat((result, rows))

In [13]:
result[['Image', 'Id']].to_csv("data/oversampled.csv", index=False)

In [14]:
result.shape

(64681, 3)

In [15]:
result.head()

,Image,Id,sightings
7,0031c258.jpg,new_whale,NaN
19,007c3603.jpg,new_whale,NaN
20,00863b8c.jpg,new_whale,NaN
26,00aa021c.jpg,new_whale,NaN
30,00ad9219.jpg,new_whale,NaN
